En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:

* Cargaremos los dos ficheros de datos
* Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    * USA
    * Australia
    * New Zealand
    * South Africa
    * Papua New Guinea

* Del dataframe de los datos climáticos seleccionaremos todas las columnas.
* Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    * Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
* Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

```python
# os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.
​
​
# El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos, es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 
clima.dtypes
​
timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object
​
# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 
​
import ast
​
clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)
​
# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 
​
​
x = clima['wind_profile'].apply(pd.Series)
​
​
# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df['rh_profile'].apply(pd.Series) 
​
# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)
​
# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 
​
# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 
​
    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 
​
    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )
​
    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)
​
# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 
```

* Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

##### Importamos librerías

In [7]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
from datetime import datetime

pd.options.display.max_columns=None

##### Cargamos dataframes

In [5]:
df_attacks = pd.read_csv('datos/attacks_limpieza_completa.csv', index_col =0)

In [6]:
df_clima = pd.read_csv('datos/datos_clima_paises.csv', index_col =0)

##### Seleccionamos los paises que nos interesan del df attacks

In [8]:
df_attacks.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


In [15]:
lista_paises = list(df_clima['pais'].unique())
lista_paises = [pais.lower() for pais in lista_paises]
lista_paises

['papua new guinea', 'new zealand', 'south africa', 'australia', 'usa']

In [16]:
df_paises = df_attacks[df_attacks['country'].isin(lista_paises)]
df_paises

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925
5,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6210,0,Unprovoked,south africa,27.0,Unespecific,Jul,Y,M,-28.816624,24.991639,HS,0,0,1,0,0,1,3,6,7,6.313925
6212,0,Unprovoked,usa,27.0,Unespecific,Unknown,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,14,7,6.313925
6213,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.776109,134.755000,HS,0,0,1,0,0,1,2,14,7,6.313925
6214,0,Unprovoked,australia,27.0,Tiger shark,Unknown,Y,M,-24.776109,134.755000,HS,0,0,1,0,0,1,2,14,7,6.313925


##### Unimos los dataframes

In [19]:
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud,fecha
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 115, 'speed':...",26,6,9,1011,none,0,0,110,3,Papua New Guinea,-5.681607,144.248908,2022-09-14 12:40:24.131455
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 115, 'speed':...",26,6,9,1014,none,0,0,110,3,Papua New Guinea,-5.681607,144.248908,2022-09-14 12:40:24.131455


In [21]:
df_clima.shape

(320, 20)

In [18]:
df_paises.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.783730,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629
3,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925
5,2018,Unprovoked,australia,27.0,Unespecific,Jun,N,M,-24.776109,134.755000,HS,1,0,0,1,0,0,3,7,7,6.313925


In [22]:
df_paises.shape

(4335, 21)